In [1]:
import pymongo
import bson
client = pymongo.MongoClient("localhost", 27017)
db = client["examples"]
inventory1 = db["inventory1"]
zips = client["zips"]["examples"]

In [2]:
inventory1_data = [
    { "_id" : bson.objectid.ObjectId("5fb55fad75b4eb537dc3d993"), "item" : "journal", "qty" : 25, "size" : {"h" : 14, "w" : 21, "uom" : "cm" }, "status" : "A"}, 
    { "_id" : bson.objectid.ObjectId("5fb55fad75b4eb537dc3d994"), "item" : "notebook", "qty" : 50, "size" : { "h" : 8.5, "w" : 11, "uom" : "in" }, "status" : "A" }, 
    { "_id" : bson.objectid.ObjectId("5fb55fad75b4eb537dc3d995"), "item" : "paper", "qty" : 100, "size" : { "h" : 8.5, "w" : 11, "uom" : "in" }, "status" : "D" },
    { "_id" : bson.objectid.ObjectId("5fb55fad75b4eb537dc3d996"), "item" : "planner", "qty" : 75, "size" : { "h" : 22.85, "w" : 30, "uom" : "cm" }, "status" : "D" }, 
    { "_id" : bson.objectid.ObjectId("5fb55fad75b4eb537dc3d997"), "item" : "postcard", "qty" : 45, "size" : {"h" : 10, "w" : 15.25, "uom" : "cm" }, "status" : "A" } 
]
inventory1.insert_many(inventory1_data)

InsertManyResult([ObjectId('5fb55fad75b4eb537dc3d993'), ObjectId('5fb55fad75b4eb537dc3d994'), ObjectId('5fb55fad75b4eb537dc3d995'), ObjectId('5fb55fad75b4eb537dc3d996'), ObjectId('5fb55fad75b4eb537dc3d997')], acknowledged=True)

In [3]:
list(inventory1.find({"status": "D"}))

[{'_id': ObjectId('5fb55fad75b4eb537dc3d995'),
  'item': 'paper',
  'qty': 100,
  'size': {'h': 8.5, 'w': 11, 'uom': 'in'},
  'status': 'D'},
 {'_id': ObjectId('5fb55fad75b4eb537dc3d996'),
  'item': 'planner',
  'qty': 75,
  'size': {'h': 22.85, 'w': 30, 'uom': 'cm'},
  'status': 'D'}]

In [ ]:
list(inventory1.find({"qty" : {"$gte" : 50} }))

In [ ]:
list(inventory1.find({"status" : "D", "qty" : {"$gte" : 50} }))

In [ ]:
list(inventory1.find({"$or": [{"status" : "D"}, {"qty" : {"$lt" : 30}}]}))

In [ ]:
# exact match of nested document, including ordering of fields 
print(list(inventory1.find( {"size": { "h": 14, "w": 21, "uom": "cm"}})))
print("===== Trying a different order of fields within")
print(list(inventory1.find( {"size": { "w": 21, "h": 14, "uom": "cm" }})))

In [ ]:
# querying a nested field
# Note: when using . notation for sub-fields, expression must be in quotes
# Also note: binary operator handled via a nested document
list(inventory1.find ( { "size.uom" : "cm", "size.h" : {"$gt" : 14 }}))

In [ ]:
inventory2 = db["inventory2"]
inventory2.insert_many([
{ "item": "journal", "instock": [ { "loc": "A", "qty": 5 }, { "loc": "C", "qty": 15 } ], "tags": ["blank", "red"], "dim": [ 14, 21 ] },
{ "item": "notebook", "instock": [ { "loc": "C", "qty": 5 } ], "tags": ["red", "blank"] , "dim": [ 14, 21 ]},
{ "item": "paper", "instock": [ { "loc": "A", "qty": 60 }, { "loc": "B", "qty": 15 } ], "tags": ["red", "blank", "plain"] , "dim": [14, 21]},
{ "item": "planner", "instock": [ { "loc": "A", "qty": 40 }, { "loc": "B", "qty": 5 } ], "tags": ["blank", "red"], "dim": [ 22.85, 30 ] },
{ "item": "postcard", "instock": [ {"loc": "B", "qty": 15 }, { "loc": "C", "qty": 35 } ], "tags": ["blue"] , "dim": [ 10, 15.25 ]}
])

In [ ]:
print("-----------")
print("Exact match of array")
print(list(inventory2.find( { "tags": ["red", "blank"] } )))

print("-----------")
print("If one of the elements matches red...")
print(list(inventory2.find( { "tags": "red" } )))

print("-----------")
print("If one matches red, one matches plain")
print(list(inventory2.find( { "tags": "red", "tags": "plain" } )))

print("-----------")
print("If one element is >15 and another is <20")
print(list(inventory2.find( { "dim": { "$gt": 15, "$lt": 20 } } )))

print("-----------")
print("If a single element is >15 and <20")
print(list(inventory2.find( { "dim": {"$elemMatch": { "$gt": 15, "$lt": 20 } } } )))

print("-----------")
print("If second item > 25..")
print("Notice again that we use quotes to when using . notation")
print(list(inventory2.find( { "dim.1": { "$gt": 25 } } )))

In [ ]:
# Arrays of Documents
print("-----------")
print("Exact match of document [like nested doc/atomic array case]")
print(list(inventory2.find( { "instock": { "loc": "A", "qty": 5 } } )))
print("-----------")
print("One nested doc has >= 20")
print(list(inventory2.find( { "instock.qty": { "$gte" : 20 } } )))
print("-----------")
print("First nested doc has >= 20")
print(list(inventory2.find( { "instock.0.qty": { "$gte" : 20 } } )))
print("-----------")
print("One doc has  20 >= qty >10")
print(list(inventory2.find( { "instock": { "$elemMatch": { "qty": { "$gt": 10, "$lte": 20 } } } } )))
print("-----------")
print("One doc has  20 >= qty, another has qty>10")
print(list(inventory2.find( { "instock.qty": { "$gt": 10, "$lte": 20 } } )))

In [ ]:
list(inventory2.find({}, {"item": 1, "instock.loc": 1, "_id": 0}))

In [ ]:
list(inventory2.find( { } ).limit(1))

In [ ]:
list(zips.find({}).limit(3))

In [ ]:
## Find states with population > 15M, sort by decending order
list(zips.aggregate( [
{ "$group": { "_id": "$state", "totalPop": { "$sum": "$pop" } } },
{ "$match": { "totalPop": { "$gte": 15000000 } } },
{ "$sort" : { "totalPop" : -1 } }
 ] ))

In [ ]:
## Find average city population per state
list(zips.aggregate( [
{ "$group": { "_id": { "state": "$state", "city": "$city" }, "pop": { "$sum": "$pop" } } },
{ "$group": { "_id": "$_id.state", "avgCityPop": { "$avg": "$pop" } } }
] ))

#with temp as (
#   select state, city, sum(pop) as pop2
#    from zips
#   group by state, city)
#select state, avg(pop2)
#from temp
#gropu by state;

In [ ]:
## Find, for every state, the biggest city and its population
list(zips.aggregate( [ 
{ "$group": { "_id": { "state": "$state", "city": "$city" }, "pop": { "$sum": "$pop" } } },
{ "$sort": { "pop": -1 } },
{ "$group": { "_id" : "$_id.state", "bigCity": { "$first": "$_id.city" }, "bigPop": { "$first": "$pop" } } },
{ "$sort" : {"bigPop" : -1} }
] ))

In [ ]:
# If we only want to keep the state and city 
list(zips.aggregate([ 
{ "$group": { "_id": { "state": "$state", "city": "$city" }, "pop": { "$sum": "$pop" } } },
{ "$sort": { "pop": -1 } },
{ "$group": { "_id" : "$_id.state", "bigCity": { "$first": "$_id.city" }, "bigPop": { "$first": "$pop" } } },
{ "$sort": {"bigPop" : -1} },
{ "$project": {"bigPop" : 0} }
] ))

In [ ]:
# If we wanted to nest the name of the city and population into a nested doc

list(zips.aggregate( [ 
{ "$group": { "_id": { "state": "$state", "city": "$city" }, "pop": { "$sum": "$pop" } } },
{ "$sort": { "pop": -1 } },
{ "$group": { "_id" : "$_id.state", "bigCity": { "$first": "$_id.city" }, "bigPop": { "$first": "$pop" } } },
{ "$sort" : {"bigPop" : -1} },
{ "$project": { "_id" : 0, "state" : "$_id", "bigCityDeets": { "name": "$bigCity", "pop": "$bigPop" } } }
] ))

In [ ]:
# Unwind expands an array by constructing documents one per element of the array
# Somewhat like flatMap in Spark

list(inventory2.aggregate( [
{ "$unwind" : "$tags" },
{ "$project" : {"_id" : 0, "instock": 0}}
] ))

In [ ]:
# Q: Imagine if we want to find sum of qtys across items. How would we do this?
# A common recipe in MQL queries is to unwind and then group by

list(inventory2.aggregate( [
    { "$unwind" : "$instock" },
    { "$group" : {"_id" : "$item", "totalqty" : {"$sum" : "$instock.qty"}}}
] ))

In [ ]:
# Conceptually, for each document,  find documents in other collection that join (equijoin)
# local field must match foreign field
# place each of them in an array
#
# Thus, a left outer equi-join, with the join results stored in an array

list(inventory2.aggregate( [
{ "$lookup" : {"from" : "inventory2", "localField": "instock.loc", "foreignField": "instock.loc", "as":"otheritems"}},
{ "$project" : {"_id" : 0, "tags" : 0, "dim" : 0}}
] ))

In [ ]:
# Lookup.. after some more projection
list(inventory2.aggregate([
{"$lookup" : {"from":"inventory2", "localField":"instock.loc", "foreignField":"instock.loc", "as":"otheritems"}},
{"$project" : {"_id" : 0, "tags" :0, "dim" :0, "otheritems._id":0, "otheritems.tags":0, "otheritems.dim":0, "otheritems.instock.qty":0}} ] )
    )

In [ ]:
inventory3 = db["inventory3"]
inventory3.insert_many([
{ "item": "journal", "instock": [ { "loc": "A", "qty": 5 }, { "loc": "C", "qty": 15 } ], "tags": ["blank", "red"], "dim": [ 14, 21 ] },
{ "item": "notebook", "instock": [ { "loc": "C", "qty": 5 } ], "tags": ["red", "blank"] , "dim": [ 14, 21 ]},
{ "item": "paper", "instock": [ { "loc": "A", "qty": 60 }, { "loc": "B", "qty": 15 } ], "tags": ["red", "blank", "plain"] , "dim": [14, 21]},
{ "item": "planner", "instock": [ { "loc": "A", "qty": 40 }, { "loc": "B", "qty": 5 } ], "tags": ["blank", "red"], "dim": [ 22.85, 30 ] },
{ "item": "postcard", "instock": [ {"loc": "B", "qty": 15 }, { "loc": "C", "qty": 35 } ], "tags": ["blue"] , "dim": [ 10, 15.25 ]},
{ "item": "postcard2", "instock": [ {"loc": "B", "qty": 15 }, { "loc": "C", "qty": 35 } ], "tags": [] , "dim": [ 10, 15.25 ]},
{ "item": "postcard3", "instock": [ {"loc": "B", "qty": 15 }, { "loc": "C", "qty": 35 } ], "dim": [ 10, 15.25 ]}
])

list(inventory3.aggregate( [
{ "$unwind" : "$tags" },
{ "$project" : {"_id" : 0, "instock": 0}}
] ))

In [ ]:
list(inventory3.find({}))

In [ ]:
inventory2 = db["inventory2"]

list(inventory2.find({}))